<a href="https://colab.research.google.com/github/prasadv26/mind-map-generator-crewai/blob/main/mind_map_creator_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import json
from crewai import Agent, Task, Crew, Process
from litellm import completion

#AGENTS DEFINITION

In [6]:
concept_extractor = Agent(
    role="Concept Extractor",
    goal="Extract the key concepts and subtopics from a given topic or document.",
    backstory="A detail-oriented researcher skilled at identifying central and supporting ideas from long texts.",
    verbose=True,
    allow_delegation=False,
    model="gpt-3.5-turbo"
)

hierarchy_builder = Agent(
    role="Hierarchy Builder",
    goal="Arrange extracted concepts into a clear mind map hierarchy with parent-child relationships.",
    backstory="An expert in organizing information and visualizing complex relationships.",
    verbose=True,
    allow_delegation=False,
    model="gpt-3.5-turbo"
)

json_formatter = Agent(
    role="JSON Formatter",
    goal="Transform the hierarchical mind map into a clean JSON structure usable by visualization tools.",
    backstory="A developer specialized in data formatting and tool compatibility.",
    verbose=True,
    allow_delegation=False,
    model="gpt-3.5-turbo"
)

image_prompt_generator = Agent(
    role="Image Prompt Generator",
    goal="Write a creative prompt to visualize the mind map as an image using AI art.",
    backstory="An expert in crafting detailed prompts for mind map visualization.",
    verbose=True,
    allow_delegation=False,
    model="gpt-3.5-turbo"
)

TASKS

In [7]:
concept_extract_task = Task(
    description="Given the input topic or article: {input_text}, extract all major concepts and subtopics as clearly as possible. Output as a structured bullet list.",
    agent=concept_extractor,
    expected_output="- Concept 1\n  - Subconcept 1.1\n- Concept 2\n  - Subconcept 2.1\n  - Subconcept 2.2"
)

hierarchy_build_task = Task(
    description="Using the extracted concepts, organize them into a tree-like mind map structure, nesting subtopics under their main topics using indentation.",
    agent=hierarchy_builder,
    context=[concept_extract_task],
    expected_output="Root\n  Concept 1\n    Subconcept 1.1\n  Concept 2\n    Subconcept 2.1"
)

json_format_task = Task(
    description=(
        "Convert the mind map hierarchy into clean, properly-structured JSON suitable for visualization. "
        "Use this format: {'name': 'Root', 'children': [{'name': 'Concept 1', 'children': [...]}, ...]}."
    ),
    agent=json_formatter,
    context=[hierarchy_build_task],
    expected_output='{"name": "Root", "children": [...]}'
)

image_prompt_task = Task(
    description="Write a descriptive prompt to visualize the mind map structure as an image (DO NOT attempt to generate or render an image).",
    agent=image_prompt_generator,
    context=[hierarchy_build_task],
    expected_output="A detailed prompt describing the topic and structure, suitable for a mind map image generator."
)

crew

In [8]:
crew = Crew(
    agents=[concept_extractor, hierarchy_builder, json_formatter, image_prompt_generator],
    tasks=[concept_extract_task, hierarchy_build_task, json_format_task, image_prompt_task],
    process=Process.sequential,
    verbose=True
)

In [9]:
topic_or_article = "Human Digestive System"

result = crew.kickoff(inputs={"input_text": topic_or_article})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: def91f10-3a2d-4e69-930f-ef7ed20a70a6                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Concept Extractor                                                                                       │
│                                                                                                                 │
│  Task: Given the input topic or article: Human Digestive System, extract all major concepts and subtopics as    │
│  clearly as possible. Output as a structured bullet list.                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Concept Extractor                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  - Human Digestive System                                                                                       │
│    - Overview of the Digestive System                                                                           │
│    - Function of the Digestive System                                                                           │
│  - Organs of the Digestive System                                                                               │
│    - Mouth                                                                                                      │
│      - Role of teeth                                                                                            │
│      - Salivary glands                                                                                          │
│    - Esophagus                                                                                                  │
│    - Stomach                                                                                                    │
│      - Gastric juices                                                                                           │
│      - Churning process                                                                                         │
│    - Small Intestine                                                                                            │
│      - Duodenum                                                                                                 │
│      - Jejunum                                                                                                  │
│      - Ileum                                                                                                    │
│      - Absorption of nutrients                                                                                  │
│    - Large Intestine                                                                                            │
│      - Cecum                                                                                                    │
│      - Colon                                                                                                    │
│        - Ascending colon                                                                                        │
│        - Transverse colon                                                                                       │
│        - Descending colon                                                                                       │
│        - Sigmoid colon                                                                                          │
│      - Rectum                                                                                                   │
│      - Anus                                                                                                     │
│  - Digestive Processes                                                                                          │
│    - Ingestion                                                                                                  │
│    - Digestion                                                                                                  │
│      - Mechanical digestion                                                                                     │
│      - Chemical digestion                              

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7e3a6f1b-d982-4253-b8ba-e28dc360fe31                                                                     │
│  Agent: Concept Extractor                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Hierarchy Builder                                                                                       │
│                                                                                                                 │
│  Task: Using the extracted concepts, organize them into a tree-like mind map structure, nesting subtopics       │
│  under their main topics using indentation.                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c1c6397b-7714-4bf2-9db3-88229bd402bc                                                                     │
│  Agent: JSON Formatter                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Image Prompt Generator                                                                                  │
│                                                                                                                 │
│  Task: Write a descriptive prompt to visualize the mind map structure as an image (DO NOT attempt to generate   │
│  or render an image).                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Image Prompt Generator                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Create a mind map visualizing the Human Digestive System, featuring a central bubble labeled "Human Digestive  │
│  System" surrounded by multiple branches representing key categories. The first branch is "Overview of the      │
│  Digestive System" leading to a sub-branch labeled "Function of the Digestive System." Next, include a branch   │
│  titled "Organs of the Digestive System" connecting to sub-branches like "Mouth," which itself has two further  │
│  sub-branches for "Role of teeth" and "Salivary glands." Continue with the branches "Esophagus," "Stomach"      │
│  (with its own sub-branches for "Gastric juices" and "Churning process"), "Small Intestine" (breaking down      │
│  into "Duodenum," "Jejunum," "Ileum," and "Absorption of nutrients"), and "Large Intestine" (leading to         │
│  sub-branches for "Cecum," "Colon" with its further subdivisions: "Ascending colon," "Transverse colon,"        │
│  "Descending colon," and "Sigmoid colon," as well as branches for "Rectum" and "Anus").                         │
│                                                                                                                 │
│  Next, depict a branch for "Digestive Processes" including "Ingestion," "Digestion" (with further subdivisions  │
│  of "Mechanical digestion" and "Chemical digestion"), "Absorption" (divided into "Nutrient absorption" and      │
│  "Water absorption"), and "Elimination."                                                                        │
│                                                                                                                 │
│  Then, add a branch for "Accessory Organs," containing sub-branches for "Liver" (with "Role in metabolism" and  │
│  "Production of bile"), "Gallbladder" (with "Storage of bile"), and "Pancreas" (further dividing into           │
│  "Production of digestive enzymes" and "Regulation of blood sugar").                                            │
│                                                                                                                 │
│  Include a branch labeled "Digestive Health," with sub-branches for "Common digestive disorders," "Importance   │
│  of dietary fiber," and "Role of hydration in digestion."                                                       │
│                                                                                                                 │
│  Another branch is "Digestive Enzymes," with sub-branches for "Types of digestive enzymes" and "Function in     │
│  digestion."                                                                                                    │
│                                                                                                                 │
│  Finally, create a branch titled "Regulation of Digestion," which includes "Hormonal control" and "Nervous      │
│  system involvement." Use various colors and icons to differentiate between categories and functions, ensuring  │
│  that the mind map is clear, organized, and visually engaging.                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 215172e0-addb-4ee7-9e6a-ee765fc90fcb                                                                     │
│  Agent: Image Prompt Generator                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: def91f10-3a2d-4e69-930f-ef7ed20a70a6                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Create a mind map visualizing the Human Digestive System, featuring a central bubble labeled     │
│  "Human Digestive System" surrounded by multiple branches representing key categories. The first branch is      │
│  "Overview of the Digestive System" leading to a sub-branch labeled "Function of the Digestive System." Next,   │
│  include a branch titled "Organs of the Digestive System" connecting to sub-branches like "Mouth," which        │
│  itself has two further sub-branches for "Role of teeth" and "Salivary glands." Continue with the branches      │
│  "Esophagus," "Stomach" (with its own sub-branches for "Gastric juices" and "Churning process"), "Small         │
│  Intestine" (breaking down into "Duodenum," "Jejunum," "Ileum," and "Absorption of nutrients"), and "Large      │
│  Intestine" (leading to sub-branches for "Cecum," "Colon" with its further subdivisions: "Ascending colon,"     │
│  "Transverse colon," "Descending colon," and "Sigmoid colon," as well as branches for "Rectum" and "Anus").     │
│                                                                                                                 │
│  Next, depict a branch for "Digestive Processes" including "Ingestion," "Digestion" (with further subdivisions  │
│  of "Mechanical digestion" and "Chemical digestion"), "Absorption" (divided into "Nutrient absorption" and      │
│  "Water absorption"), and "Elimination."                                                                        │
│                                                                                                                 │
│  Then, add a branch for "Accessory Organs," containing sub-branches for "Liver" (with "Role in metabolism" and  │
│  "Production of bile"), "Gallbladder" (with "Storage of bile"), and "Pancreas" (further dividing into           │
│  "Production of digestive enzymes" and "Regulation of blood sugar").                                            │
│                                                                                                                 │
│  Include a branch labeled "Digestive Health," with sub-branches for "Common digestive disorders," "Importance   │
│  of dietary fiber," and "Role of hydration in digestion."                                                       │
│                                                                                                                 │
│  Another branch is "Digestive Enzymes," with sub-branches for "Types of digestive enzymes" and "Function in     │
│  digestion."                                                                                                    │
│                                                                                                                 │
│  Finally, create a branch titled "Regulation of Digestion," which includes "Hormonal control" and "Nervous      │
│  system involvement." Use various colors and icons to differentiate between categories and functions, ensuring  │
│  that the mind map is clear, organized, and visually engaging.                                                  │
│                                                                                                                 │
│                                                       